In [40]:
import pandas as pd 
import numpy as np
import re
import matplotlib as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
# pd.set_option('display.float_format', '{:,.0f}'.format)

In [41]:
sal = pd.read_csv('data/salary.csv')

In [42]:
# convert HireDate to datetime
sal['HireDate'] = pd.to_datetime(sal['HireDate'], unit='ms')
# sal['HireDate'] = sal['HireDate'].dt.strftime('%m%d%Y')

# clean FiscalYear
sal['FiscalYear'] = sal['FiscalYear'].str.replace('^FY', '', regex=True)

# drop rows with missing values in GrossPay
sal = sal.dropna(subset=['GrossPay'])

# removes the trailing number in parentheses from AgencyName
sal['AgencyName'] = sal['AgencyName'].str.replace(r'\s\(\d+\)', '', regex=True)

# strip spaces 
sal[sal.select_dtypes('object').columns] = sal[sal.select_dtypes('object').columns].apply(lambda x: x.str.strip())

# List of words to replace
replace_dict = {
    'Police': 'Police Department',
    'Fire': 'Fire Department',
}

# Loop through the dictionary and replace each word with regex
for old, new in replace_dict.items():
    sal['AgencyName'] = sal['AgencyName'].apply(lambda x: re.sub(rf'\b{re.escape(old)}\b(?! Department)', new, str(x)) if isinstance(x, str) else x)

# Step 1: Fill JobTitle with AgencyName where JobTitle is NaN or empty if no agency name drops
sal['JobTitle'] = sal['JobTitle'].fillna(sal['AgencyName'])
sal['JobTitle'] = sal['JobTitle'].apply(lambda x: x if isinstance(x, str) and x.strip() != "" else sal['AgencyName'])
sal = sal[sal['AgencyName'].notna() & (sal['AgencyName'].str.strip() != "")]

# Remove rows where AnnualSalary is NaN or 0
sal = sal[sal['AnnualSalary'].notna() & (sal['AnnualSalary'] != 0)]

# Remove rows where GrossPay is NaN or 0
sal = sal[sal['GrossPay'].notna() & (sal['GrossPay'] != 0)]

# Discrepancy in amount
sal['Pay_Discrepancy'] = sal['GrossPay'] - sal['AnnualSalary']

# Discrepancy in percentage (relative to AnnualSalary)
sal['Pay_Discrepancy_Pct'] = ((sal['GrossPay'] - sal['AnnualSalary']) / sal['AnnualSalary']) * 100



In [43]:
## exports annual sal == 0 and gross pay > 60000
# annual_sal_err = sal[(sal['AnnualSalary'] == 0) & (sal['GrossPay'] > 60000)]
# annual_sal_err
# annual_sal_err.to_csv('check_into/annual_sal_err.csv', index=False)


In [44]:
# # filter by year 

# def filter_by_year(df, year):
#     """
#     Filters the DataFrame by the given year.
    
#     Parameters:
#     df (DataFrame): The DataFrame to filter.
#     year (int): The year to filter by.
    
#     Returns:
#     DataFrame: The filtered DataFrame.
#     """
#     return df[df['FiscalYear'] == str(year)]

# # Example usage:
# year = 2024
# filtered_2024 = filter_by_year(sal, year)

In [45]:

# def agency_breakdown(filtered_2024):
#     department = filtered_2024.groupby('AgencyName').agg({
#     'AnnualSalary': 'sum',
#     'GrossPay': 'sum',
#     }).reset_index()

#     # Now calculate discrepancies after aggregation
#     department['Pay_Discrepancy'] = department['GrossPay'] - department['AnnualSalary']
#     department['Pay_Discrepancy_Pct'] = (department['Pay_Discrepancy'] / department['AnnualSalary']) * 100

#     department = department.sort_values(by='Pay_Discrepancy', ascending=False)
    
#     return department

# agency_breakdown(filtered_2024)

In [46]:
sal.head(1)

,ID,Name,JobTitle,AgencyID,AgencyName,HireDate,AnnualSalary,GrossPay,FiscalYear,Pay_Discrepancy,Pay_Discrepancy_Pct
0,1,"Aaron,Patricia G",Facilities/Office Services II,A03,OED-Employment Dev,1979-10-24,50845.0,45505.94,2011,-5339.06,-10.500659


In [47]:
# code for 5
def plot_info(year, data) -> None:
    # Ensure 'FiscalYear' is treated as a string (you can also change this to integer if needed)
    data['FiscalYear'] = data['FiscalYear'].astype(str)

    # Filter the data for the given year
    year_data = data[data['FiscalYear'] == str(year)]

    # If no data for that year, exit function
    if year_data.empty:
        print(f"No data available for {year}")
        return

    # Group and aggregate data by FiscalYear
    summary = year_data.groupby('FiscalYear')[['GrossPay', 'AnnualSalary']].sum().reset_index()

    # Extract actual and expected values
    actual = summary['GrossPay'].iloc[0]
    expected = summary['AnnualSalary'].iloc[0]

    # Calculate the discrepancy (absolute and percentage)
    discrepancy = actual - expected
    discrepancy_pct = (discrepancy / expected) * 100

    # Create the gauge plot
    steps = [
        {'range': [0, expected], 'color': '#004080'}  # Dark blue for expected salary
    ]
    
    # Add yellow step if actual exceeds expected
    if actual > expected:
        steps.append({'range': [expected, actual], 'color': 'yellow'})  # Yellow for actual salary
    
    fig = go.Figure(go.Indicator(
        mode="gauge+number+delta",
        value=actual,
        delta={
            'reference': expected,
            'increasing': {'color': "red"},
            'decreasing': {'color': "green"}
        },
        gauge={
            'axis': {'range': [0, expected * 1.1]},  
            'bar': {'color': 'rgba(0,0,0,0)'},
            'steps': steps,
            'threshold': {
                'line': {'color': "red", 'width': 4},  
                'thickness': .95,
                'value': actual  # Place the red line at the actual
            }
        },
        title={'text': f"{year} Budgeted Salary Spend vs. Salary Spend Actual", 'font': {'size': 20}},
    ))

    fig.update_layout(
        title_font={'size': 20},  # Set font size for the overall layout title
    )

    # Display the discrepancy
    # print(f"Discrepancy for {year}: ${discrepancy:,.2f} ({discrepancy_pct:.2f}%)")

    # Show the figure
    fig.show()

# Example usage:
plot_info(2024, sal)

In [48]:
sal.head(1)

,ID,Name,JobTitle,AgencyID,AgencyName,HireDate,AnnualSalary,GrossPay,FiscalYear,Pay_Discrepancy,Pay_Discrepancy_Pct
0,1,"Aaron,Patricia G",Facilities/Office Services II,A03,OED-Employment Dev,1979-10-24,50845.0,45505.94,2011,-5339.06,-10.500659


In [49]:
# code for 7
def calculate_total_spend(year, data) -> str:
    # Filter the data for the specified year
    year_filter = data[data['FiscalYear'] == str(year)]
    
    # Calculate the sum of 'YTD_Total' for the filtered data
    total_spend = year_filter['GrossPay'].sum().round(2)

    # Formats thousands separators
    total_spend = "{:,.2f}".format(total_spend)
    
    # Return the total spend as a float
    return str(total_spend)

# Example usage:
year = 2024
total_spend = calculate_total_spend(year, sal)
total_spend

'1,020,010,972.66'

In [50]:
# code for 9 
def calculate_total_budget(year, data) -> str:
    # Filter the data for the specified year
    year_filter = data[data['FiscalYear'] == str(year)]
    
    # Calculate the sum of 'Salary_Total' for the filtered data
    total_spend = year_filter['AnnualSalary'].sum().round(2)

    # Formats thousands separators
    total_spend = "{:,.2f}".format(total_spend)
    
    # Return the total spend as a float
    return str(total_spend)

# Example usage:
year = 2024
total_budget = calculate_total_budget(year, sal)
total_budget

'881,629,451.94'

In [51]:
#code for 12
def calculate_budget_difference(year, data) -> str:
    # Filter the data for the specified year
    year_filter = data[data['FiscalYear'] == str(year)]
    
    if year_filter.empty:
        return "No data for the specified year."
    
    # Extract actual and expected values
    actual_spend = year_filter['GrossPay'].sum()
    budgeted_salary = year_filter['AnnualSalary'].sum()

    # Calculate the percentage difference
    difference = ((actual_spend - budgeted_salary) / budgeted_salary) * 100
    
    # Format the difference with a '+' or '-' and thousands separators
    return f"{difference:+,.2f}%"

# Example usage:
year = 2024
budget_difference = calculate_budget_difference(year, sal)
budget_difference

'+15.70%'

In [53]:
# code for 10
def top_emp_dev(year, data):
    # we had to avoid division by zero and only calculate Discrepancy_Percent for employees with Salary_Total >= 20k
    # Filter the data for the given year
    top_employee = data[data['FiscalYear'] == str(year)].copy()

    # Round the Discrepancy_Percent to 2 decimal places
    top_employee['Pay_Discrepancy_Pct'] = top_employee['Pay_Discrepancy_Pct'].round(2)

    # handle Salary_Total < 20k
    top_employee = top_employee[top_employee['AnnualSalary'] >= 10000]

    # Sort the DataFrame by 'Discrepancy_Percent' in descending order
    top_employee = top_employee.sort_values(by='Pay_Discrepancy', ascending=False)

    # Keep only the specified columns
    top_employee = top_employee[['FiscalYear', 'Name', 'AgencyName', 'JobTitle',
                                 'AnnualSalary', 'GrossPay',
                                 'Pay_Discrepancy', 'Pay_Discrepancy_Pct']]
    
    # Reset index and drop the old index column
    top_employee.reset_index(drop=True, inplace=True)

    return top_employee.head(20)

# Example usage:
year = 2024
top_employee = top_emp_dev(year, sal)
top_employee

,FiscalYear,Name,AgencyName,JobTitle,AnnualSalary,GrossPay,Pay_Discrepancy,Pay_Discrepancy_Pct
0,2024,"Lunsford,David",Fire Department,"62712 - Paramedic, EMT-P",113157.85,358586.44,245428.59,216.89
1,2024,"Harris,William",Police Department,41112 - Police Sergeant,117663.00,324820.37,207157.37,176.06
2,2024,"Wallace,Antonio",Fire Department,"41209 - EMT Firefighter, Suppression",98769.35,290218.16,191448.81,193.83
3,2024,"Booker,Jammie",Public Works - Water & Waste Water,53516 - WWW Utilities Installer Repairer Super...,86404.98,268233.11,181828.13,210.44
4,2024,"Pettis,Dennis",Fire Department,"41268 - Fire Lieutenant, Safety and Risk Manag...",110128.59,285677.16,175548.57,159.40
5,2024,"Mcmillian,Paul",Police Department,41112 - Police Sergeant,119984.00,295443.17,175459.17,146.24
6,2024,"Whitted,Lakeshia",Police Department,41112 - Police Sergeant,118895.00,292142.04,173247.04,145.71
7,2024,"Rondholz,Christopher",Fire Department,"62712 - Paramedic, EMT-P",103159.97,276072.62,172912.65,167.62
8,2024,"Parker,Leon",Fire Department,41211 - Firefighter Suppression,105158.93,273163.65,168004.72,159.76
9,2024,"Lofton,Osiris",Police Department,41111 - Police Officer,99687.00,265123.51,165436.51,165.96


In [55]:
# code for 11
def department_discrepancy(year, data):
    # Group by 'CalYear' and 'Department', summing 'GrossPay' and 'AnnualSalary'
    department = data.groupby(['FiscalYear', 'AgencyName'])[['GrossPay', 'AnnualSalary']].sum().reset_index()

    # Filter the data by the given year
    department = department[department['FiscalYear'] == str(year)]

    # Calculate the percentage difference: ((GrossPay - AnnualSalary) / AnnualSalary) * 100
    department['Pay_Discrepancy_Pct'] = ((department['GrossPay'] - department['AnnualSalary']) / department['AnnualSalary']) * 100

    # Round the Pay_Discrepancy_Pct to 2 decimal places
    department['Pay_Discrepancy_Pct'] = department['Pay_Discrepancy_Pct'].round(2)

    # Sort the DataFrame by 'Pay_Discrepancy_Pct' in ascending order
    department = department.sort_values(by='Pay_Discrepancy_Pct', ascending=False)

    # Format 'GrossPay' and 'AnnualSalary' with thousands separators
    department['GrossPay'] = department['GrossPay'].apply(lambda x: f"{x:,.2f}")
    department['AnnualSalary'] = department['AnnualSalary'].apply(lambda x: f"{x:,.2f}")

    # Rename columns for final output
    department = department.rename(columns={
        'GrossPay': 'Total Salary Spend',
        'AnnualSalary': 'Salary Budget',
        'Pay_Discrepancy_Pct': 'Discrepancy Percent'
    })

    # Return the result
    return department

# Example usage:
year = 2024
department = department_discrepancy(year, sal)
department

,FiscalYear,AgencyName,Total Salary Spend,Salary Budget,Discrepancy Percent
870,2024,Environmental Control Board,"575,234.50","291,987.73",97.01
904,2024,Recreation & Parks - Recreation,"11,804,606.66","6,407,503.74",84.23
907,2024,SCS - Special City Services,"140,000.64","80,000.00",75.00
901,2024,Rec & Parks,"755,060.15","525,899.88",43.57
908,2024,Sheriff's Office,"18,567,076.97","13,355,700.62",39.02
896,2024,Police Department,"299,080,950.52","219,924,247.65",35.99
912,2024,Transportation - Highways (weekly),"2,450,226.69","1,837,576.90",33.34
877,2024,Fire Department,"175,657,307.62","138,157,902.38",27.14
914,2024,Transportation - Traffic,"10,362,753.17","8,160,165.22",26.99
899,2024,Public Works - Solid Waste (weekly),"10,708,659.18","8,478,894.67",26.30
